<img src="mioti.png" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura Data preprocessing</center>

# Challenge S6: Reglas de asociación para análisis de cesta de la compra

## Objetivos

El objetivo de este challenge es enfrentarse a un problema de generación de reglas de asociación real: a partir de datos de transacciones de compra de un supermercado, realizar un estudio de la cesta de la compra (*market basket analysis*) mediante la generación de reglas de asociación.

In [1]:
# Descomentar esto para instalar paquete mlxtend
#import sys
#!{sys.executable} -m pip install mlxtend

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import pandas as pd

## Carga de datos

Para comenzar, cargamos el fichero de tickets de venta:

In [2]:
df = pd.read_excel('data/Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Y lo convertimos al formato esperado por la librería de reglas de asociación

In [3]:
# Preprocesamiento y filtrado de columnas no útiles
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

# Filtramos por país, agrupamos por ticket y transponemos
basket = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [4]:
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Convertimos los binarios a enteros
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [6]:
basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [7]:
basket_sets.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


* ¿Cuál es el tamaño de nuestro conjunto de items $I$ ?

1694

* ¿ Cuál es el tamaño de nuestro conjunto de transacciones $D$ ?

5

## Reglas de asociación con mlxtend

Ahora vamos a utilizar la librería [`mlxtend`](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/) para generar las reglas de asociación más interesantes en nuestro conjunto de datos. 

* Comenzamos calculando el soporte de los itemsets frecuentes de nuestro conjunto, aplicando un umbral mínimo de 0.07:

In [10]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [13]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.102845,(6 RIBBONS RUSTIC CHARM)
1,0.070022,(ALARM CLOCK BAKELIKE PINK)
2,0.072210,(GUMBALL COAT RACK)
3,0.091904,(JAM MAKING SET PRINTED)
4,0.078775,(JUMBO BAG RED RETROSPOT)
5,0.100656,(JUMBO BAG WOODLAND ANIMALS)
6,0.078775,(LUNCH BAG WOODLAND)
7,0.085339,(PACK OF 72 RETROSPOT CAKE CASES)
8,0.115974,(PLASTERS IN TIN CIRCUS PARADE)
9,0.107221,(PLASTERS IN TIN SPACEBOY)


* Prueba a ejecutar el mismo algoritmo apriori con umbral mínimo de soporte de 0.05 y 0.09. ¿Qué observas? ¿A qué crees que se debe?

In [14]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.102845,(6 RIBBONS RUSTIC CHARM)
1,0.070022,(ALARM CLOCK BAKELIKE PINK)
2,0.072210,(GUMBALL COAT RACK)
3,0.091904,(JAM MAKING SET PRINTED)
4,0.078775,(JUMBO BAG RED RETROSPOT)
5,0.100656,(JUMBO BAG WOODLAND ANIMALS)
6,0.078775,(LUNCH BAG WOODLAND)
7,0.085339,(PACK OF 72 RETROSPOT CAKE CASES)
8,0.115974,(PLASTERS IN TIN CIRCUS PARADE)
9,0.107221,(PLASTERS IN TIN SPACEBOY)


In [15]:
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.102845,(6 RIBBONS RUSTIC CHARM)
1,0.070022,(ALARM CLOCK BAKELIKE PINK)
2,0.065646,(CHARLOTTE BAG APPLES DESIGN)
3,0.050328,(CHILDRENS CUTLERY DOLLY GIRL)
4,0.061269,(COFFEE MUG APPLES DESIGN)
5,0.063457,(FAWN BLUE HOT WATER BOTTLE)
6,0.072210,(GUMBALL COAT RACK)
7,0.056893,(IVORY KITCHEN SCALES)
8,0.063457,(JAM JAR WITH PINK LID)
9,0.091904,(JAM MAKING SET PRINTED)


In [16]:
frequent_itemsets = apriori(basket_sets, min_support=0.09, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.102845,(6 RIBBONS RUSTIC CHARM)
1,0.091904,(JAM MAKING SET PRINTED)
2,0.100656,(JUMBO BAG WOODLAND ANIMALS)
3,0.115974,(PLASTERS IN TIN CIRCUS PARADE)
4,0.107221,(PLASTERS IN TIN SPACEBOY)
5,0.137856,(PLASTERS IN TIN WOODLAND ANIMALS)
6,0.096280,(RED TOADSTOOL LED NIGHT LIGHT)
7,0.137856,(REGENCY CAKESTAND 3 TIER)
8,0.157549,(ROUND SNACK BOXES SET OF 4 FRUITS)
9,0.245077,(ROUND SNACK BOXES SET OF4 WOODLAND)


De ahora en adelante utilizaremos el primer soporte calculado con un umbral de valor 0.07. 

* ¿Cuál es el item con mayor valor de soporte?

(ROUND SNACK BOXES SET OF4 WOODLAND)

In [18]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.102845,(6 RIBBONS RUSTIC CHARM)
1,0.070022,(ALARM CLOCK BAKELIKE PINK)
2,0.072210,(GUMBALL COAT RACK)
3,0.091904,(JAM MAKING SET PRINTED)
4,0.078775,(JUMBO BAG RED RETROSPOT)
5,0.100656,(JUMBO BAG WOODLAND ANIMALS)
6,0.078775,(LUNCH BAG WOODLAND)
7,0.085339,(PACK OF 72 RETROSPOT CAKE CASES)
8,0.115974,(PLASTERS IN TIN CIRCUS PARADE)
9,0.107221,(PLASTERS IN TIN SPACEBOY)


In [20]:
frequent_itemsets.sort_values(by='support')

,support,itemsets
12,0.070022,(RED RETROSPOT CHARLOTTE BAG)
18,0.070022,(SET OF 3 REGENCY CAKE TINS)
13,0.070022,(RED RETROSPOT CUP)
10,0.070022,(PLASTERS IN TIN STRONGMAN)
24,0.070022,"(ROUND SNACK BOXES SET OF4 WOODLAND, SPACEBOY ..."
1,0.070022,(ALARM CLOCK BAKELIKE PINK)
2,0.072210,(GUMBALL COAT RACK)
22,0.074398,"(ROUND SNACK BOXES SET OF4 WOODLAND, PLASTERS ..."
6,0.078775,(LUNCH BAG WOODLAND)
20,0.078775,(STRAWBERRY LUNCH BOX WITH CUTLERY)


* Ahora vamos a calcular las reglas de asociación utilizando el soporte calculado previamente. Prueba utilizando la métrica confidence con un umbral mínimo de 1.

In [23]:
as_conf = association_rules(frequent_itemsets, metric="confidence", min_threshold=1)
as_conf

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


* Ahora prueba utilizando la métrica lift con un umbral mínimo de 1.

In [24]:
as_lift = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
as_lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ROUND SNACK BOXES SET OF4 WOODLAND),(PLASTERS IN TIN WOODLAND ANIMALS),0.245077,0.137856,0.074398,0.303571,2.202098,0.040613,1.237951
1,(PLASTERS IN TIN WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.137856,0.245077,0.074398,0.539683,2.202098,0.040613,1.640006
2,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.245077,0.157549,0.131291,0.535714,3.400298,0.092679,1.814509
3,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540
4,(ROUND SNACK BOXES SET OF4 WOODLAND),(SPACEBOY LUNCH BOX),0.245077,0.102845,0.070022,0.285714,2.778116,0.044817,1.256018
5,(SPACEBOY LUNCH BOX),(ROUND SNACK BOXES SET OF4 WOODLAND),0.102845,0.245077,0.070022,0.680851,2.778116,0.044817,2.365427


* Sobre estas últimas reglas calculadas, filtra aquellas que tengan un lift mayor que 1.5 y un confidence mayor que 0.7

In [48]:
mask = (as_lift['lift'] > 1.5) & (as_lift['confidence'] > 0.7)
as_lift[mask]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.52954


* ¿Cómo intepretarías el significado en concreto de estas reglas de asociación generadas?

 Para estas reglas, se puede concluir que existen pocos items que tengan dependencia entre sí; además de que las que existen asociación no hay un 100% de certeza de esa asociación. 
 
 Esto también implica que los productos pueden tener en efecto negativo el uno sobre el otro.

## Reglas de asociación a mano

Tomemos como ejemplo la regla de asociación generada en el paso previo. Vamos a realizar los cálculos previamente obtenidos por `mlxtend`, ahora nosotros a mano. 

* Comenzamos por el soporte. Calcula el soporte del antecedente, del consecuente y de ambos, de la última regla de asociación (sin utilizar mlxtend).

In [51]:
sum(basket_sets['ROUND SNACK BOXES SET OF 4 FRUITS'])/len(basket_sets)

0.1575492341356674

In [52]:
sum(basket_sets['ROUND SNACK BOXES SET OF4 WOODLAND'])/len(basket_sets)

0.24507658643326038

* Una vez calculado los soportes, calcula la confianza (confidence) de la regla.

In [56]:
0.131291/(sum(basket_sets['ROUND SNACK BOXES SET OF 4 FRUITS'])/len(basket_sets))

0.8333331527777776

* Ahora calcula el lift.

In [57]:
0.131291/((sum(basket_sets['ROUND SNACK BOXES SET OF 4 FRUITS'])/len(basket_sets))*sum(basket_sets['ROUND SNACK BOXES SET OF4 WOODLAND'])/len(basket_sets))

3.400296882316468

Comprueba que obtienes los mismos resultados que con `mlxtend`.